In [1]:
import tensorflow as tf

#ㅔ라스의 내장 데이터셋에서 mnist 데이터셋 로드
mnist = tf.keras.datasets.mnist

#load_data()로 데이터셋 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 정규화
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()


In [2]:
#책에 중략으로 코드가 빠진것을 확인 - 모델 아무거나 불러옴
# 모델정의 
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    # 노드=10개가 되어야 한다
    tf.keras.layers.Dense(10, activation="softmax"),
])

#모델 컴파일
model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics = ["accuracy"])

In [3]:
# 체크포인트 설정
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = "tmp_checkpoint.ckpt",
                                               save_weights_only = True,
                                               save_best_only = True,
                                               monitor = "val_loss",
                                               verbose=1)

In [4]:
model.fit(x_train, y_train,
         validation_data=(x_test,y_test),
         epochs=10,
         callbacks=[checkpoint]
         )

Epoch 1/10
1865/1875 [============================>.] - ETA: 0s - loss: 0.2324 - accuracy: 0.9307
Epoch 00001: val_loss improved from inf to 0.11862, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2320 - accuracy: 0.9309 - val_loss: 0.1186 - val_accuracy: 0.9646
Epoch 2/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0962 - accuracy: 0.9697
Epoch 00002: val_loss improved from 0.11862 to 0.08548, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0962 - accuracy: 0.9697 - val_loss: 0.0855 - val_accuracy: 0.9732
Epoch 3/10
1867/1875 [============================>.] - ETA: 0s - loss: 0.0657 - accuracy: 0.9797
Epoch 00003: val_loss improved from 0.08548 to 0.07941, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0658 - accuracy: 0.9797 - val_loss: 0.0794 - val_accuracy: 0.9756
Epoch 4/10
1856/1875 [============

In [5]:
"""
#모델을 파일로 저장
훈련 종료하여 가중치가 업데이트된 모델 인스턴스를 저장할 수 있다.
save() 메소드를 사용하는데, save()메소드를 호출할 때 저장할 파일의 디렉토리를 포함하는 파일명을 매개변수로 지정한다
모델을 저장되는 형식은 HDF5 포맷과 SavedModel 포맷 두 가지가 있다

모델을 저장할 파일명이 .h5 확장자를 포함하는 경우는 모델이 HDF%(Hierarchical Data Fromat)포맷으로 저장한다
HDF5 파일 형식은 대용량의 데이터를 저장하기 위한 파일 형식
"""
# 모델을 h5 포맷으로 저장
model.save("h5-model.h5")

In [6]:
"""
# 저장된 모델 복원
tensorflow.keras.models.load_model()메소드로 저장된 모델을 복원할 수 있다.
저장된 HDF5 파일 형식의 모델을 복원하여 summary()를 확인하면 모델 구조가 그대로 복원되어 있다
"""

#모델 복원
h5_model = tf.keras.models.load_model("h5-model.h5")
h5_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 219,818
Trainable params: 219,818
Non-trainable params: 0
_________________________________________________________________


In [7]:
"""
복원한 모델을 검증하여 복원이 잘 되었는지 확인한다. 검증 셋을 evaluate 함수에 대입해, 손실함수와 평가지표를 계산한다
"""
# 모델 검증
loss, acc = h5_model.evaluate(x_test, y_test, verbose=0)
print(f"h5 model] loss: {loss:.5f}, acc:{acc:5f}")

h5 model] loss: 0.09809, acc:0.977600


In [8]:
"""
h5 확장자를 생략하면 SavedModel 포맷으로 저장되며 텐서플로우2에서 기본으로 지원하는 파일 포맷이다.
SavedModel 포맷으로 저장된 모델을 복원할 때 HDF5와 같은 방식으로 복원하면 된다
"""
# 모델을 SavedModel 포맷으로 저장
model.save("saved-model")

INFO:tensorflow:Assets written to: saved-model\assets


In [9]:
"""
SavedModel 포맷으로 저장된 모델을 복원하여 summary()확인한다
"""
# 모델 복원
saved_model = tf.keras.models.load_model("saved-model")
saved_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 219,818
Trainable params: 219,818
Non-trainable params: 0
_________________________________________________________________


In [10]:
"""
SavedModel 형식으로 저장 및 복원된 모델의 성능을 확인하면, 
HDF5형식으로 저장 및 복원된 모델과 동일한 성능을 나타낸다
"""
# 모델 검증(HDF5 포맷)
loss, acc = h5_model.evaluate(x_test, y_test, verbose=0)
print(f"h5 model- loss:{loss:5f}, acc:{acc:5f}")

# 모델 검증 (SavedModel 포맷)
loss, acc = saved_model.evaluate(x_test, y_test, verbose=0)
print(f"Saved model- loss:{loss:5f}, acc:{acc:5f}")


h5 model- loss:0.098094, acc:0.977600
Saved model- loss:0.098094, acc:0.977600
